In [1]:
# ─── CHALLENGE 1 ────────────────────────────────────────────────────────────────

import re
from itertools import product

with open('input14') as file:
    lines = file.read().splitlines()

class Decoder():
    def __init__(self):
        # Most values of 36-bit memory space are unused / have value 0. Therefore,
        # a memory object with sparse capability is needed. Here we use a dict
        # to save to memory.

        self.memory1 = dict()
        self.memory2 = dict()

    def set_mask(self, mask):
        self.mask = mask

    def step1(self, address, value):
        # apply inputs according to challenge 1.
        def apply_mask_to_value(mask, value_binary):
            new_value_binary = []
            for c_mask, c_string in zip(mask, value_binary):
                new_char = c_string if c_mask == 'X' else c_mask
                new_value_binary.append(new_char)
            return ''.join(new_value_binary)
        value_binary = format(int(value), 'b').zfill(36)
        new_value_binary = apply_mask_to_value(self.mask, value_binary)
        self.memory1[address] = int(new_value_binary, 2)
        
    def step2(self, address, value):
        # apply inputs according to challenge 2.
        def apply_mask_to_address(mask, address_binary):
            new_value_binary = []
            x_counter = 0
            for c_mask, c_string in zip(mask, address_binary):
                # x from mask is dominant
                if c_mask == 'X':
                    new_char = chr(x_counter+65)
                    x_counter += 1
                # 1 from mask is dominant
                if c_mask == '1':
                    new_char = c_mask
                # 0 from mask is not dominant
                if c_mask == '0':
                    new_char = c_string
                new_value_binary.append(new_char)
            return ''.join(new_value_binary), x_counter

        address_binary = format(int(address), 'b').zfill(36)
        new_address_binary_template, x_counter = apply_mask_to_address(self.mask, address_binary)
        combinations = product('01', repeat=x_counter)
        
        # apply each combination of Xs in mask and write to memory
        for combination in combinations:
            new_address_binary = new_address_binary_template
            for i, val in enumerate(combination, start=65):
                new_address_binary = new_address_binary.replace(chr(i), val)
            self.memory2[new_address_binary] = int(value)

    def eval(self, challenge):
        if challenge == 1:
            counters = map(lambda x: x[1], self.memory1.items())
        if challenge == 2:
            counters = map(lambda x: x[1], self.memory2.items())
        return sum(counters)


decoder = Decoder()
        
for line in lines:
    if line.startswith('mask'):
        new_mask = re.search('\S{36}', line)
        decoder.set_mask(new_mask[0])
    else:
        address, value = re.findall('([\d]+)', line)
        decoder.step1(address, value)
        decoder.step2(address, value)
        
print('result 1: ', decoder.eval(challenge=1))
print('result 2: ', decoder.eval(challenge=2))

result 1:  9296748256641
result 2:  4877695371685
